In [1]:
import lightgbm as lgb
import numpy as np
import pandas as pd
from tqdm.auto import tqdm

from metric import apk, mapk
from utils import plot_images
import catboost

In [2]:
def objective(trial):
    num_popular_items = trial.suggest_int('num_popular_items', 12, 36)
    num_repurchase_week = trial.suggest_int('num_repurchase_week', 4, 16)
    num_cooc_week = trial.suggest_int('num_cooc_week', 4, 16)
    num_train_weeks = trial.suggest_int('num_train_weeks', 1, 4)
    volume_week = trial.suggest_int('volume_week', 1, 4)
    cooc_prob = trial.suggest_uniform('cooc_prob', 0.02, 0.05)
    transaction_dynamic_week = trial.suggest_int('transaction_dynamic_week', 1, 8)

    dataset = '100'

    transactions = pd.read_pickle(f"input/{dataset}/transactions_train.pkl")
    users = pd.read_pickle(f"input/{dataset}/users.pkl")
    items = pd.read_pickle(f"input/{dataset}/items.pkl")

    def create_candidates(transactions: pd.DataFrame, target_users: np.ndarray, week: int) -> pd.DataFrame:
        """
        transactions
            original transactions (user, item, week)
        target_users, week
            候補生成対象のユーザー
            weekで指定されている週の段階での情報のみから作られる
        """
        assert len(target_users) == len(set(target_users))

        def create_candidates_repurchase(
                strategy: str,
                transactions: pd.DataFrame,
                target_users: np.ndarray,
                week_start: int,
                week_end: int) -> pd.DataFrame:
            tr = transactions.query("user in @target_users and @week_start <= week <= @week_end")[['user', 'item', 'week']].drop_duplicates(ignore_index=True)

            gr_week = tr.groupby(['user', 'item'])['week'].min().reset_index(name='week')
            gr_volume = tr.groupby(['user', 'item']).size().reset_index(name='volume')

            gr_week['week_rank'] = gr_week.groupby('user')['week'].rank()
            gr_volume['volume_rank'] = gr_volume.groupby('user')['volume'].rank()

            candidates = gr_week.merge(gr_volume, on=['user', 'item'])
            candidates = candidates[['user', 'item', 'week_rank', 'volume_rank']].rename(columns={'week_rank': f'{strategy}_week_rank', 'volume_rank': f'{strategy}_volume_rank'})

            candidates['strategy'] = strategy
            return candidates

        def create_candidates_popular(
                strategy: str,
                transactions: pd.DataFrame,
                target_users: np.ndarray,
                week_start: int,
                week_end: int) -> pd.DataFrame:
            tr = transactions.query("@week_start <= week <= @week_end")[['user', 'item']].drop_duplicates(ignore_index=True)
            popular_items = tr['item'].value_counts().index.values[:num_popular_items]
            popular_items = pd.DataFrame({
                'item': popular_items,
                'rank': range(num_popular_items),
                'crossjoinkey': 1,
            })

            candidates = pd.DataFrame({
                'user': target_users,
                'crossjoinkey': 1,
            })

            candidates = candidates.merge(popular_items, on='crossjoinkey').drop('crossjoinkey', axis=1)
            candidates = candidates.rename(columns={'rank': f'{strategy}_rank'})

            candidates['strategy'] = strategy
            return candidates

        def create_candidates_cooc(
                strategy: str,
                transactions: pd.DataFrame,
                week_start: int,
                week_end: int,
                base_candidates: pd.DataFrame) -> pd.DataFrame:
            hoge = cooc_prob
            tr = transactions.query("@week_start <= week == @week_end")[['user', 'item', 'week']].drop_duplicates(ignore_index=True)
            tr = tr.merge(tr.rename(columns={'item': 'item_with', 'week': 'week_with'}), on='user').query("item != item_with and week <= week_with")[['item', 'item_with']].reset_index(drop=True)
            gr_sz = tr.groupby('item').size().reset_index(name='tot')
            gr_cnt = tr.groupby(['item', 'item_with']).size().reset_index(name='cnt')
            item2item = gr_cnt.merge(gr_sz, on='item')
            item2item['ratio'] = item2item['cnt'] / item2item['tot']
            item2item = item2item.query("ratio >= @hoge").reset_index(drop=True)

            base_candidates_columns = [c for c in base_candidates.columns if '_' in c]
            replace = {c: f"{strategy}_{c}" for c in base_candidates_columns}
            candidates = base_candidates.merge(item2item, on='item').drop(['item', 'cnt'], axis=1).rename(columns={'item_with': 'item'}).rename(columns=replace)
            candidates = candidates.rename(columns={'ratio': f'{strategy}_ratio', 'tot': f'{strategy}_tot'})

            candidates['strategy'] = strategy
            return candidates

        def create_candidates_same_product_code(
                strategy: str,
                items: pd.DataFrame,
                base_candidates: pd.DataFrame) -> pd.DataFrame:
            item2item = items[['item', 'product_code']].merge(items[['item', 'product_code']].rename({'item': 'item_with'}, axis=1), on='product_code')[['item', 'item_with']].query("item != item_with").reset_index(drop=True)

            base_candidates_columns = [c for c in base_candidates.columns if '_' in c]
            replace = {c: f"{strategy}_{c}" for c in base_candidates_columns}
            candidates = base_candidates.merge(item2item, on='item').drop('item', axis=1).rename(columns={'item_with': 'item'}).rename(columns=replace)

            candidates['strategy'] = strategy
            return candidates

        candidates_repurchase = create_candidates_repurchase('repurchase', transactions, target_users, week, week + num_repurchase_week)
        candidates_popular = create_candidates_popular('pop', transactions, target_users, week, week)
        candidates_cooc = create_candidates_cooc('cooc', transactions, week, week + num_cooc_week, candidates_repurchase)
        candidates_same_product_code = create_candidates_same_product_code('same_product_code', items, candidates_repurchase)

        candidates = [
            candidates_repurchase,
            candidates_popular,
            candidates_cooc,
            candidates_same_product_code,
        ]

        candidates = pd.concat(candidates).reset_index(drop=True)

        volumes = candidates.groupby('strategy').size().reset_index(name='volume').sort_values(by='volume', ascending=False).reset_index(drop=True)
        volumes['ratio'] = volumes['volume'] / volumes['volume'].sum()

        return candidates

    # valid: week=1
    # train: week=2..1+num_train_weeks
    candidates = []
    for week in range(1, 2+num_train_weeks):
        target_users = transactions.query("week == @week")['user'].unique()
        candidates.append(create_candidates(transactions, target_users, week+1))
    def merge_labels(candidates: pd.DataFrame, week: int) -> pd.DataFrame:
        """
        candidatesに対してweekで指定される週のトランザクションからラベルを付与する
        """
        labels = transactions[transactions['week'] == week][['user', 'item']].drop_duplicates(ignore_index=True)
        labels['y'] = 1
        original_positives = len(labels)
        labels = candidates.merge(labels, on=['user', 'item'], how='left')
        labels['y'] = labels['y'].fillna(0)

        remaining_positives_total = labels[['user', 'item', 'y']].drop_duplicates(ignore_index=True)['y'].sum()
        recall = remaining_positives_total / original_positives

        volumes = candidates.groupby('strategy').size().reset_index(name='volume')
        remaining_positives = labels.groupby('strategy')['y'].sum().reset_index()
        remaining_positives = remaining_positives.merge(volumes, on='strategy')
        remaining_positives['recall'] = remaining_positives['y'] / original_positives
        remaining_positives['hit_ratio'] = remaining_positives['y'] / remaining_positives['volume']
        remaining_positives = remaining_positives.sort_values(by='y', ascending=False).reset_index(drop=True)

        return labels

    for idx in range(len(candidates)):
        candidates[idx] = merge_labels(candidates[idx], 1+idx)

    def get_query_group(df):
        def run_length_encoding(sequence):
            comp_seq_index, = np.concatenate(([True], sequence[1:] != sequence[:-1], [True])).nonzero()
            return sequence[comp_seq_index[:-1]], np.ediff1d(comp_seq_index)
        users = df['user'].values
        _, group = run_length_encoding(users)
        return list(group)

    def drop_trivial_users(labels):
        """
        LightGBMのxendgcやlambdarankでは正例のみや負例のみのuserは学習に無意味なのと、メトリックの計算がおかしくなるので省く
        """
        bef = len(labels)
        df = labels[labels['user'].isin(labels[['user', 'y']].drop_duplicates().groupby('user').size().reset_index(name='sz').query("sz==2").user)].reset_index(drop=True)
        aft = len(df)
        return df

    for idx in range(len(candidates)):
        candidates[idx]['week'] = 1+idx

    valid_all = candidates[0].copy()

    for idx in range(len(candidates)):
        candidates[idx] = drop_trivial_users(candidates[idx])

    def attach_features(transactions: pd.DataFrame, users: pd.DataFrame, items: pd.DataFrame, candidates: pd.DataFrame, week: int) -> pd.DataFrame:
        """
        user, itemに対して特徴を横付けする
        week: これを含めた以前の情報は使って良い
        """
        n_original = len(candidates)
        df = candidates.copy()

        # user static features
        user_features = ['FN', 'Active', 'age', 'club_member_status_idx', 'fashion_news_frequency_idx']
        df = df.merge(users[['user'] + user_features], on='user')

        # item static features
        item_features = [c for c in items.columns if c.endswith('idx')]
        df = df.merge(items[['item'] + item_features], on='item')

        # user dynamic features (transactions)
        num_weeks = transaction_dynamic_week
        week_end = week + num_weeks
        tmp = transactions.query("@week <= week < @week_end").groupby('user')[['price', 'sales_channel_id']].agg(['mean', 'std'])
        tmp.columns = ['user_' + '_'.join(a) for a in tmp.columns.to_flat_index()]
        df = df.merge(tmp, on='user', how='left')

        # item dynamic features (transactions)
        num_weeks = transaction_dynamic_week
        week_end = week + num_weeks
        tmp = transactions.query("@week <= week < @week_end").groupby('item')[['price', 'sales_channel_id']].agg(['mean', 'std'])
        tmp.columns = ['item_' + '_'.join(a) for a in tmp.columns.to_flat_index()]
        df = df.merge(tmp, on='item', how='left')

        # item dynamic features (user features)
        num_weeks = transaction_dynamic_week
        week_end = week + num_weeks
        tmp = transactions.query("@week <= week < @week_end").merge(users[['user', 'age']], on='user')
        tmp = tmp.groupby('item')['age'].agg(['mean', 'std'])
        tmp.columns = [f'age_{a}' for a in tmp.columns.to_flat_index()]
        df = df.merge(tmp, on='item', how='left')

        # item freshness features
        tmp = transactions.query("@week <= week").groupby('item')['week'].min().reset_index(name='item_week_min')
        tmp['item_week_min'] -= week
        df = df.merge(tmp, on='item', how='left')

        # item volume features
        num_weeks = volume_week
        week_end = week + num_weeks
        tmp = transactions.query("@week <= week < @week_end").groupby('item').size().reset_index(name='item_volume')
        df = df.merge(tmp, on='item', how='left')

        # # user freshness features
        # tmp = transactions.query("@week <= week").groupby('user')['week'].min().reset_index(name='user_week_min')
        # tmp['user_week_min'] -= week
        # df = df.merge(tmp, on='user', how='left')

        # # user volume features
        # num_weeks = 1
        # week_end = week + num_weeks
        # tmp = transactions.query("@week <= week < @week_end").groupby('user').size().reset_index(name='user_volume')
        # df = df.merge(tmp, on='user', how='left')

        # user-item freshness features
        tmp = transactions.query("@week <= week").groupby(['user', 'item'])['week'].min().reset_index(name='user_item_week_min')
        tmp['user_item_week_min'] -= week
        df = df.merge(tmp, on=['item', 'user'], how='left')

        # user-item volume features
        num_weeks = volume_week
        week_end = week + num_weeks
        tmp = transactions.query("@week <= week < @week_end").groupby(['user', 'item']).size().reset_index(name='user_item_volume')
        df = df.merge(tmp, on=['user', 'item'], how='left')

        assert len(df) == n_original
        return df


    valid_all = attach_features(transactions, users, items, valid_all, 2)
    for idx in range(len(candidates)):
        candidates[idx] = attach_features(transactions, users, items, candidates[idx], 2+idx)

    for idx in range(len(candidates)):
        candidates[idx]['group'] = candidates[idx]['week'].astype(str) + '_' + candidates[idx]['user'].astype(str)
        candidates[idx] = candidates[idx].sort_values(by='group').reset_index(drop=True)

    valid = candidates[0]

    def concat_train(datasets, begin, num):
        train = pd.concat([datasets[idx] for idx in range(begin, begin+num)])
        return train

    train = concat_train(candidates, 1, num_train_weeks)
    feature_columns = [c for c in valid.columns if c not in ['y', 'strategy', 'week', 'group']]


    cat_feature_values = [c for c in feature_columns if c.endswith('idx')]
    cat_features = [feature_columns.index(c) for c in cat_feature_values]

    train_pool = catboost.Pool(data=train[feature_columns], label=train['y'], group_id=train['group'], cat_features=cat_features)
    valid_pool = catboost.Pool(data=valid[feature_columns], label=valid['y'], group_id=valid['group'], cat_features=cat_features)

    params = {
        'loss_function': 'YetiRank',
        'use_best_model': True,
        'one_hot_max_size': 300,
        'metric_period': 100,
    }
    model = catboost.CatBoost(params)
    model.fit(train_pool, eval_set=valid_pool)

    valid_all['pred'] = model.predict(valid_all[feature_columns])
    pred = valid_all.groupby(['user', 'item'])['pred'].max().reset_index()

    pred = pred.sort_values(by=['user', 'pred'], ascending=False).reset_index(drop=True).groupby('user')['item'].apply(lambda x: list(x)[:12]).reset_index()
    pred

    gt = transactions.query("week == 1").groupby('user')['item'].apply(list).reset_index().rename(columns={'item': 'gt'})
    merged = gt.merge(pred, on='user', how='left')
    merged['item'] = merged['item'].fillna('').apply(list)
    merged

    return mapk(merged['gt'], merged['item'])


In [3]:
import optuna

study = optuna.create_study(
    direction='maximize',
    study_name='cat',
)
study.optimize(objective, timeout=43200)
study.trials_dataframe().sort_values(by='value',ascending=False).reset_index(drop=True).to_csv('cat.csv', index=False)

[I 2022-04-07 00:30:07,182] A new study created in memory with name: cat


0:	test: 0.2507958	best: 0.2507958 (0)	total: 917ms	remaining: 15m 15s
100:	test: 0.3641860	best: 0.3641860 (100)	total: 1m 22s	remaining: 12m 17s
200:	test: 0.3743213	best: 0.3743213 (200)	total: 2m 43s	remaining: 10m 51s
300:	test: 0.3783455	best: 0.3783455 (300)	total: 4m 3s	remaining: 9m 26s
400:	test: 0.3767817	best: 0.3783455 (300)	total: 5m 23s	remaining: 8m 3s
500:	test: 0.3782076	best: 0.3783455 (300)	total: 6m 43s	remaining: 6m 42s
600:	test: 0.3790387	best: 0.3790387 (600)	total: 8m 2s	remaining: 5m 20s
700:	test: 0.3799666	best: 0.3799666 (700)	total: 9m 22s	remaining: 3m 59s
800:	test: 0.3800420	best: 0.3800420 (800)	total: 10m 41s	remaining: 2m 39s
900:	test: 0.3806844	best: 0.3806844 (900)	total: 12m 1s	remaining: 1m 19s
999:	test: 0.3808476	best: 0.3808476 (999)	total: 13m 19s	remaining: 0us

bestTest = 0.3808475695
bestIteration = 999



[I 2022-04-07 00:46:24,969] Trial 0 finished with value: 0.029444172949849443 and parameters: {'num_popular_items': 29, 'num_repurchase_week': 11, 'num_cooc_week': 13, 'num_train_weeks': 2, 'volume_week': 3, 'cooc_prob': 0.03675601065850931, 'transaction_dynamic_week': 5}. Best is trial 0 with value: 0.029444172949849443.


0:	test: 0.2397908	best: 0.2397908 (0)	total: 291ms	remaining: 4m 50s
100:	test: 0.4401510	best: 0.4401510 (100)	total: 28.2s	remaining: 4m 10s
200:	test: 0.4511399	best: 0.4511399 (200)	total: 55.8s	remaining: 3m 41s
300:	test: 0.4552388	best: 0.4552388 (300)	total: 1m 23s	remaining: 3m 13s
400:	test: 0.4528181	best: 0.4552388 (300)	total: 1m 50s	remaining: 2m 45s
500:	test: 0.4542032	best: 0.4552388 (300)	total: 2m 17s	remaining: 2m 17s
600:	test: 0.4553187	best: 0.4553187 (600)	total: 2m 44s	remaining: 1m 49s
700:	test: 0.4562548	best: 0.4562548 (700)	total: 3m 12s	remaining: 1m 21s
800:	test: 0.4566884	best: 0.4566884 (800)	total: 3m 38s	remaining: 54.4s
900:	test: 0.4574690	best: 0.4574690 (900)	total: 4m 5s	remaining: 27s
999:	test: 0.4579576	best: 0.4579576 (999)	total: 4m 32s	remaining: 0us

bestTest = 0.4579576394
bestIteration = 999



[I 2022-04-07 00:52:59,275] Trial 1 finished with value: 0.028885310530626526 and parameters: {'num_popular_items': 14, 'num_repurchase_week': 11, 'num_cooc_week': 10, 'num_train_weeks': 1, 'volume_week': 3, 'cooc_prob': 0.04710093671387566, 'transaction_dynamic_week': 5}. Best is trial 0 with value: 0.029444172949849443.


...

[I 2022-04-07 12:08:25,012] Trial 29 finished with value: 0.0299532290870617 and parameters: {'num_popular_items': 20, 'num_repurchase_week': 14, 'num_cooc_week': 12, 'num_train_weeks': 2, 'volume_week': 1, 'cooc_prob': 0.037014624407833055, 'transaction_dynamic_week': 7}. Best is trial 14 with value: 0.030083009807994422.


0:	test: 0.2316993	best: 0.2316993 (0)	total: 1.05s	remaining: 17m 32s
100:	test: 0.3588199	best: 0.3588199 (100)	total: 1m 43s	remaining: 15m 21s
200:	test: 0.3721389	best: 0.3721389 (200)	total: 3m 25s	remaining: 13m 36s
300:	test: 0.3759958	best: 0.3759958 (300)	total: 5m 6s	remaining: 11m 51s
400:	test: 0.3795038	best: 0.3795038 (400)	total: 6m 46s	remaining: 10m 7s
500:	test: 0.3815588	best: 0.3815588 (500)	total: 8m 26s	remaining: 8m 24s
600:	test: 0.3826871	best: 0.3826871 (600)	total: 10m 6s	remaining: 6m 42s
700:	test: 0.3835722	best: 0.3835722 (700)	total: 11m 45s	remaining: 5m 1s
800:	test: 0.3844007	best: 0.3844007 (800)	total: 13m 25s	remaining: 3m 20s
900:	test: 0.3850671	best: 0.3850671 (900)	total: 15m 4s	remaining: 1m 39s
999:	test: 0.3857228	best: 0.3857228 (999)	total: 16m 42s	remaining: 0us

bestTest = 0.3857227676
bestIteration = 999



[I 2022-04-07 12:28:35,620] Trial 30 finished with value: 0.029677413780709868 and parameters: {'num_popular_items': 28, 'num_repurchase_week': 11, 'num_cooc_week': 8, 'num_train_weeks': 3, 'volume_week': 1, 'cooc_prob': 0.04242487488121538, 'transaction_dynamic_week': 4}. Best is trial 14 with value: 0.030083009807994422.


0:	test: 0.2403486	best: 0.2403486 (0)	total: 996ms	remaining: 16m 35s
100:	test: 0.3942409	best: 0.3942409 (100)	total: 1m 39s	remaining: 14m 43s
200:	test: 0.4074185	best: 0.4074185 (200)	total: 3m 17s	remaining: 13m 4s
300:	test: 0.4129960	best: 0.4129960 (300)	total: 4m 54s	remaining: 11m 24s
400:	test: 0.4159684	best: 0.4159684 (400)	total: 6m 31s	remaining: 9m 44s
500:	test: 0.4171273	best: 0.4171273 (500)	total: 8m 7s	remaining: 8m 5s
600:	test: 0.4172772	best: 0.4172772 (600)	total: 9m 44s	remaining: 6m 28s
700:	test: 0.4175934	best: 0.4175934 (700)	total: 11m 21s	remaining: 4m 50s
800:	test: 0.4179654	best: 0.4179654 (800)	total: 12m 58s	remaining: 3m 13s
900:	test: 0.4187493	best: 0.4187493 (900)	total: 14m 35s	remaining: 1m 36s
999:	test: 0.4194287	best: 0.4194287 (999)	total: 16m 11s	remaining: 0us

bestTest = 0.41942873
bestIteration = 999



[I 2022-04-07 12:47:47,628] Trial 31 finished with value: 0.02975309477829758 and parameters: {'num_popular_items': 20, 'num_repurchase_week': 14, 'num_cooc_week': 12, 'num_train_weeks': 2, 'volume_week': 1, 'cooc_prob': 0.03761711650453397, 'transaction_dynamic_week': 7}. Best is trial 14 with value: 0.030083009807994422.


In [5]:
import pickle
with open("study.pkl", 'wb') as f:
    pickle.dump(study, f)

In [18]:
optuna.visualization.plot_contour(study)

In [7]:
optuna.visualization.plot_param_importances(study)

In [8]:
optuna.visualization.plot_optimization_history(study)

In [20]:
df_params = study.trials_dataframe().sort_values(by='value',ascending=False).reset_index(drop=True)
df_params.head(10)

,number,value,datetime_start,datetime_complete,duration,params_cooc_prob,params_num_cooc_week,params_num_popular_items,params_num_repurchase_week,params_num_train_weeks,params_transaction_dynamic_week,params_volume_week,state
0,14,0.030083,2022-04-07 07:34:00.609792,2022-04-07 07:52:42.598176,0 days 00:18:41.988384,0.031043,11,18,13,2,8,1,COMPLETE
1,18,0.030025,2022-04-07 08:21:02.339974,2022-04-07 08:41:02.496876,0 days 00:20:00.156902,0.032152,11,19,14,2,8,1,COMPLETE
2,27,0.029979,2022-04-07 11:10:15.077520,2022-04-07 11:30:05.140806,0 days 00:19:50.063286,0.034363,10,19,15,2,8,1,COMPLETE
3,29,0.029953,2022-04-07 11:48:43.236487,2022-04-07 12:08:25.012343,0 days 00:19:41.775856,0.037015,12,20,14,2,7,1,COMPLETE
4,3,0.029869,2022-04-07 01:16:27.955465,2022-04-07 01:28:26.802021,0 days 00:11:58.846556,0.031006,10,15,16,1,8,1,COMPLETE
5,24,0.029869,2022-04-07 10:26:16.683892,2022-04-07 10:48:23.206783,0 days 00:22:06.522891,0.034036,9,22,15,2,8,1,COMPLETE
6,20,0.029842,2022-04-07 08:54:11.553717,2022-04-07 09:24:56.352621,0 days 00:30:44.798904,0.032936,11,26,14,3,6,1,COMPLETE
7,11,0.029809,2022-04-07 04:17:06.904653,2022-04-07 05:37:49.794505,0 days 01:20:42.889852,0.020389,16,20,16,4,3,2,COMPLETE
8,10,0.029794,2022-04-07 03:03:32.226862,2022-04-07 04:17:06.903409,0 days 01:13:34.676547,0.021659,16,20,16,4,3,2,COMPLETE
9,31,0.029753,2022-04-07 12:28:35.621183,2022-04-07 12:47:47.628187,0 days 00:19:12.007004,0.037617,12,20,14,2,7,1,COMPLETE
